## 0. Importação das bibliotecas

In [7]:
import pandas as pd
import os 

## 1. Carregando os dados

In [10]:
df_prices = pd.read_csv('../data/raw/teste_indicium_precificacao.csv')


## 1. Descrição dos dados

In [11]:
## Visualizando as colunas do dataset
df_prices.columns

Index(['id', 'nome', 'host_id', 'host_name', 'bairro_group', 'bairro',
       'latitude', 'longitude', 'room_type', 'price', 'minimo_noites',
       'numero_de_reviews', 'ultima_review', 'reviews_por_mes',
       'calculado_host_listings_count', 'disponibilidade_365'],
      dtype='object')

## 1.1. Dimensão dos dados

In [12]:
## Verificando o nº de linhas e colunas
print('Número de linhas:{}'.format(df_prices.shape[0]))
print('Número de colunas:{}'.format(df_prices.shape[1]))

Número de linhas:48894
Número de colunas:16


## 1.2. Tipo de dados

In [13]:

## Verificando os tipos de dados, e, se necessário, mudar os tipos não adequados
df_prices.dtypes

id                                 int64
nome                              object
host_id                            int64
host_name                         object
bairro_group                      object
bairro                            object
latitude                         float64
longitude                        float64
room_type                         object
price                              int64
minimo_noites                      int64
numero_de_reviews                  int64
ultima_review                     object
reviews_por_mes                  float64
calculado_host_listings_count      int64
disponibilidade_365                int64
dtype: object

## 1.3. Checando valores nulos

In [14]:
## Verificando e Tratando os valores nulos
df_prices.isna().sum()

id                                   0
nome                                16
host_id                              0
host_name                           21
bairro_group                         0
bairro                               0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimo_noites                        0
numero_de_reviews                    0
ultima_review                    10052
reviews_por_mes                  10052
calculado_host_listings_count        0
disponibilidade_365                  0
dtype: int64

In [ ]:
## O tratamento adequado para os dados NA 
## Para colunas nome e host_name, os valores nulos serão preenchidos por 'Desconhecido'; para valores de ultima_review serão 
# substituídos por uma data padrão e as reviews em branco serão substituídas por 0 

df_prices = df_prices.assign(
    nome=df_prices['nome'].fillna('Desconhecido'),
    host_name=df_prices['host_name'].fillna('Desconhecido'),
    ultima_review=df_prices['ultima_review'].fillna(pd.Timestamp('2000-01-01')),
    reviews_por_mes=df_prices['reviews_por_mes'].fillna(0)
)

## Verificando se ainda há valores nulos 
print(df_prices.isna().sum())



id                               0
nome                             0
host_id                          0
host_name                        0
bairro_group                     0
bairro                           0
latitude                         0
longitude                        0
room_type                        0
price                            0
minimo_noites                    0
numero_de_reviews                0
ultima_review                    0
reviews_por_mes                  0
calculado_host_listings_count    0
disponibilidade_365              0
dtype: int64
